In [1]:
%load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths


import sys, os
sys.path.append(os.path.abspath('../..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.7.6, build 905165378
Copyright (C) 2014-2023 Monash University, NICTA, Data61


In [230]:
exchange = 'kucoin'
pair = 'BTC_USDT'
timeframe = '4h'

odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()

odf.shape, odf.head(5)

((7701, 8),
                          timestamp    open    high     low   close     volume   
 date                                                                            
 2020-01-01 00:00:00  1577836800000  7196.2  7243.2  7176.1  7225.4  62.743859  \
 2020-01-01 04:00:00  1577851200000  7225.4  7233.8  7203.1  7209.5  36.248625   
 2020-01-01 08:00:00  1577865600000  7209.4  7232.0  7183.0  7195.6  52.545595   
 2020-01-01 12:00:00  1577880000000  7195.7  7251.8  7195.6  7231.4  69.704852   
 2020-01-01 16:00:00  1577894400000  7231.5  7248.0  7217.1  7229.0  60.889127   
 
                                    date     idate  
 date                                               
 2020-01-01 00:00:00 2020-01-01 00:00:00  20200101  
 2020-01-01 04:00:00 2020-01-01 04:00:00  20200101  
 2020-01-01 08:00:00 2020-01-01 08:00:00  20200101  
 2020-01-01 12:00:00 2020-01-01 12:00:00  20200101  
 2020-01-01 16:00:00 2020-01-01 16:00:00  20200101  )

In [231]:
# hdf[['close']].set_axis(['h_close'],axis=1)

In [427]:
#
h_columns = ['open', 'high', 'low', 'close', 'volume']
resampling_period = '1D'
hdf = odf[h_columns].resample(resampling_period).agg(
    OrderedDict([
        ('open', 'first'),
        ('high', 'max'),
        ('low', 'min'),
        ('close', 'last'),
        ('volume', 'sum'),
    ])
)

gbsz = odf[[]].groupby(pd.Grouper(freq=resampling_period)).size()
odf['h_dtime'] = sorted([c for b in [[t]*s for t,s in gbsz.reset_index().values] for c in b])

odf = odf.join(hdf[['open','high','low','close','volume']].set_axis(['h_open','h_high','h_low','h_close','h_volume'], axis=1), on='h_dtime')
odf.head(10)

ValueError: columns overlap but no suffix specified: Index(['h_open', 'h_high', 'h_low', 'h_close', 'h_volume'], dtype='object')

In [293]:
### get dataframe of original and resampled time
smpl_df = pd.DataFrame(np.array([[k,v] for k, idxs in odf[[]].resample(resampling_period).indices.items() for v in odf.iloc[idxs].index.values])
).set_axis(['h_dtime', 'time'], axis=1).sort_values(by='time')
smpl_df


,h_dtime,time
0,2020-01-01,2020-01-01 00:00:00
1,2020-01-01,2020-01-01 04:00:00
2,2020-01-01,2020-01-01 08:00:00
3,2020-01-01,2020-01-01 12:00:00
4,2020-01-01,2020-01-01 16:00:00
...,...,...
7696,2023-07-06,2023-07-06 16:00:00
7697,2023-07-06,2023-07-06 20:00:00
7698,2023-07-07,2023-07-07 00:00:00
7699,2023-07-07,2023-07-07 04:00:00


In [433]:
hupeaks 
# np.array([[hdf.loc[iidx, 'close']] + df.index[[0,-1]].tolist() 
#                            for iidx in hupeaks
#                            for idx in [hdf[[]].iloc[iidx].index]
#                            for df in [wdf[wdf.h_dtime == iidx]]
#                         ])

# wdf

array([   9,   17,   23,   29,   39,   44,   53,   65,   70,   83,   92,
         98,  108,  117,  122,  127,  134,  139,  145,  152,  161,  167,
        173,  182,  189,  195,  200,  208,  213,  222,  229,  236,  244,
        250,  255,  262,  267,  272,  278,  285,  297,  304,  309,  317,
        328,  334,  340,  353,  360,  368,  373,  379,  387,  399,  405,
        410,  417,  427,  437,  444,  454,  460,  468,  475,  486,  493,
        499,  505,  511,  519,  525,  530,  536,  545,  550,  557,  564,
        576,  584,  590,  600,  614,  620,  626,  631,  639,  644,  653,
        658,  663,  671,  677,  683,  689,  694,  700,  706,  711,  717,
        722,  731,  742,  748,  759,  765,  770,  776,  783,  790,  798,
        803,  808,  818,  824,  829,  839,  845,  854,  860,  865,  872,
        881,  887,  896,  906,  918,  930,  939,  950,  955,  967,  974,
        985,  990,  995, 1002, 1007, 1012, 1020, 1026, 1032, 1039, 1044,
       1049, 1057, 1064, 1072, 1078, 1084, 1090, 11

In [503]:
wdf,hdf, df_hmap = None,None,None
ihupeaks, hupeak_props = None, None
ihdpeaks = None
def get_indicators(**kwargs):
    global wdf, hdf, df_hmap
    global ihupeaks, ihdpeaks #, hupeak_props, hupeaks_idxs, last_upeak_iidxs, last_upeaks

    wdf, pkwargs = kwargs['wdf'], kwargs['pkwargs']
    print('pkwargs', pkwargs)
    # wdf = pkwargs['wdf']
    inow, pkdist, pct_hpeak = pkwargs['inow'], pkwargs['pkdist'],pkwargs['pct_hpeak']

    ### resampling to higher timeframe for peaks
    h_columns = ['open', 'high', 'low', 'close', 'volume']
    hsample = pkwargs['hsample']
    hdf = wdf[h_columns].resample(hsample).agg(
        OrderedDict([
            ('open', 'first'),
            ('high', 'max'),
            ('low', 'min'),
            ('close', 'last'),
            ('volume', 'sum'),
        ])
    )
    df_hmap = pd.DataFrame(np.array([[k,v] for k, idxs in odf[[]].resample(resampling_period).indices.items() for v in odf.iloc[idxs].index.values])
).set_axis(['h_dtime', 'time'], axis=1).sort_values(by='time')
    # whdf = hdf.loc[wdf.index.date[0]:wdf.index.date[-1]]
    # ixnow = wdf.index[]
    # hix_now = wdf.iloc[inow].h_dtime.

    ihupeaks, hupeak_props = find_peaks(hdf.loc[:wdf.iloc[inow].h_dtime].iloc[:-1].close, prominence=1, distance=pkdist)
    ihdpeaks, hdpeak_props = find_peaks(-hdf.loc[:wdf.iloc[inow].h_dtime].iloc[:-1].close, prominence=1, distance=pkdist)
    # hdpeaks, hdpeak_props = find_peaks(-hdf.close, prominence=1, distance=pkdist)
    # print('upeaks: ', upeaks)
    # print('wdf:', wdf)
    # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    # results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    # hupeaks_idxs = hdf.index[hupeaks]
    # hdpeaks_idxs = hdf.index[hdpeaks]


    # last_upeak_iidxs = ([ p[-1] if p.shape[0] > 0 else pd.NA for p in [hupeaks[hupeaks < i] for i in range(hdf.shape[0])] ])
    # gwdf['upeak_close'] = 
    # last_upeak_prices = [ hdf.iloc[i]['close'] if not pd.isna(i) else pd.NA for i in last_upeak_iidxs]

    return {'wdf': wdf, 'hdf': hdf, 'df_hmap': df_hmap, 'ihupeaks': ihupeaks, 'ihdpeaks': ihdpeaks} 
# (hupeaks, hupeaks_idxs, last_upeak_iidxs, last_upeak_prices), hdpeaks_idxs, wdf, hdf

signals = None
def get_signals(indicators, pkwargs):
    global signals
    wdf = indicators['wdf']
    hdf = indicators['hdf']
    ihupeaks = indicators['ihupeaks']
    ihdpeaks = indicators['ihdpeaks']
    # (hupeaks, hupeaks_idxs, last_upeak_iidxs, last_upeak_prices), dpeaks_idx, whdf = indicators
    pct_hpeak = pkwargs['pct_hpeak']
    
    # dist_last_upeaks = ((last_upeak_prices / wdf.close) - 1).abs()
    # signals = (dist_last_upeaks >= pct_hpeak[0]) & (dist_last_upeaks <= pct_hpeak[1])

    # print('pct_hpeak', pct_hpeak)
    # upeaks, upeak_props = find_peaks(wdf.close, distance=pkdist)
    # dpeaks, dpeak_props = find_peaks(-wdf.close, distance=pkdist)

    # # results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
    # # results_full = peak_widths(wdf.close, peaks, rel_height=1)
    
    # upeaks_idxs = wdf.index[upeaks]
    # dpeaks_idxs = wdf.index[dpeaks]

    # close_peaks = wdf.loc[peaks_idx].close
    return signals #upeaks_idxs,dpeaks_idxs

In [495]:
ihupeaks

In [506]:

### Ranges
# nest_asyncio.apply()

strategy = 'swings'
strategy_params_json = f'./par-{strategy}.json'

strategy_params = {
    "pkdist": {"wdg": IntSlider(value=7,min=1,max=100,step=1,description='PKDist',continuous_update=False)},
    "hsample": {"wdg": Text(value='1D',description='hsample',continuous_update=False)},
    "pjn": {"wdg": IntSlider(value=7,min=0,max=100,step=1,description='pjn',continuous_update=False)},
    # "pkprom": {"wdg": FloatRangeSlider(value=(0.05,1),min=0,max=1,step=0.005,description='PKProm',continuous_update=False)},
    "pct_hpeak": {"wdg": FloatRangeSlider(value=(0,0.1),min=0,max=0.2,step=0.001,description='dhp',continuous_update=False, readout_format='.3f')},
}

mpl.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300, 'lines.markersize': 4})
plt.rcParams['lines.linewidth'] = 0.4

def draw_func(ax_dict, indicators, signals, pkwargs):
    axa = ax_dict['A']

    wdf = indicators['wdf']
    hdf = indicators['hdf']
    ihupeaks = indicators['ihupeaks']
    ihdpeaks = indicators['ihdpeaks']
    df_hmap = indicators['df_hmap']
    # (hupeaks, hupeaks_idxs, last_upeak_iidxs, last_upeak_prices), dpeaks_idx, whdf = indicators
    # print('indicators: ', indicators)
    # print(wdf.close[upeaks_idx])
    
    # axa.scatter(upeaks_idxs, whdf.close[upeaks_idxs], s=50, c='r', alpha=0.2)
    # axa.scatter(dpeaks_idx, whdf.close[dpeaks_idx], s=50, c='g', alpha=0.2)
    # hupeaks_idxs = hdf.index[hupeaks]
    pjn = 7
    upeak_spans = np.array([
        [hdf.loc[hidx, 'close']] + df.index[[0,-1]].tolist()                        
        for ihu in ihupeaks
        for [hidx, ehidx] in [hdf.index.values[[ihu, min(hdf.shape[0]-1, ihu+pjn)]]]
        for df in [ wdf[(wdf.h_dtime >= hidx) & (wdf.h_dtime <= ehidx) ] ]
    ])
    dpeak_spans = np.array([
        [hdf.loc[hidx, 'close']] + df.index[[0,-1]].tolist()                        
        for ihu in ihdpeaks
        for [hidx, ehidx] in [hdf.index.values[[ihu, min(hdf.shape[0]-1, ihu+pjn)]]]
        for df in [ wdf[(wdf.h_dtime >= hidx) & (wdf.h_dtime <= ehidx) ] ]
    ])

    hdf.close.plot(ax=axa, lw=2, c='m', alpha=0.3)
    # n_fu = 6
    axa.hlines(*upeak_spans.T, colors='r', linewidths=1.5, alpha=0.3)
    axa.hlines(*dpeak_spans.T, colors='g', linewidths=1.5, alpha=0.3)
    
    # xmin,xmax = np.array(upeaks_idxs, wdf.index.values)
    # axa.hlines(wdf.close, [ for i in ], wdf.index)
    # for idx in wdf.index[signals]: 
    #     for ax in [axa]: ax.axvline(idx)
    # axa.plot(dpeaks_idx, wdf.close[upeaks_idx], "^")
    

fu_printer(odf, strategy, strategy_params = strategy_params, draw_func = draw_func, indicator_func=get_indicators, signal_func=get_signals)

#     ax1.plot(peaks_idx, close_peaks, "^", c='m', markersize=2, alpha=0.5)
#     ax1.vlines(x=peaks_idx, ymin=close_peaks - peak_props["prominences"],
#            ymax = close_peaks, color = "C1", lw=0.2)
#     ax1.hlines(results_full[1:][0], xmin=wdf.index[results_full[1:][1].astype(int)], xmax=wdf.index[results_full[1:][2].astype(int)], color="C1", lw=0.3)
#     ax1.hlines(results_half[1:][0], xmin=wdf.index[results_half[1:][1].astype(int)], xmax=wdf.index[results_half[1:][2].astype(int)], color="C2", lw=0.3)
#     # ax1.hlines(y=(close_peaks - peak_props["prominences"]), xmin=wdf.index[peak_props["left_bases"]],
#     #        xmax=wdf.index[peak_props["right_bases"]], color = "C1")




Output()

In [493]:
hdf

,open,high,low,close,volume
date,,,,,
2022-09-29,19288.4,19643.5,19112.6,19588.6,4152.007101
2022-09-30,19588.7,20177.9,19155.0,19422.0,15365.244618
2022-10-01,19422.0,19483.6,19155.8,19311.5,3412.585300
2022-10-02,19311.6,19392.2,18925.0,19056.8,7201.969193
2022-10-03,19056.9,19717.6,18960.3,19628.8,11522.062496
2022-10-04,19627.1,20461.4,19489.5,20339.3,12988.010811
2022-10-05,20339.3,20364.0,19739.0,20158.2,9107.785577
2022-10-06,20158.2,20447.8,19858.0,19960.2,13089.300125
2022-10-07,19960.3,20065.0,19330.2,19532.5,9130.827588


In [481]:
ihu = 5
hdf.index.values[[ihu, ihu+5]]

array(['2020-01-27T00:00:00.000000000', '2020-02-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [291]:
### get dataframe of original and resampled time
pd.DataFrame(np.array([[k,v] for k, idxs in gwdf[[]].resample(resampling_period).indices.items() for v in gwdf.iloc[idxs].index.values])
).set_axis(['h_dtime', 'time'], axis=1).sort_values(by='time')


,h_dtime,time
0,2020-01-06,2020-01-06 08:00:00
1,2020-01-06,2020-01-06 12:00:00
2,2020-01-06,2020-01-06 16:00:00
3,2020-01-06,2020-01-06 20:00:00
4,2020-01-07,2020-01-07 00:00:00
...,...,...
59,2020-01-16,2020-01-16 04:00:00
60,2020-01-16,2020-01-16 08:00:00
61,2020-01-16,2020-01-16 12:00:00
62,2020-01-16,2020-01-16 16:00:00


In [65]:
gwdf.iloc[5]['close']

7200.2

In [76]:

peaks, peak_props = find_peaks(wdf.close, prominence=1, distance=10)
# wdf.iloc[peaks].close

results_half = peak_widths(wdf.close, peaks, rel_height=0.5)
# np.shape(results_half[1:])
results_half[1:][1].astype(int)
# fig, (ax1,ax2, ax3) = plt.subplots(3, 1, height_ratios=[3,1,1], sharex=True);

# plot_candles(wdf, ax=ax1) if candles else ax1.plot(wdf.close, lw=0.3, c='b')
# plt.plot(peaks, wdf.iloc[peaks].close, "x")

array([  1,   9,  24,  35,  49,  61,  81,  91, 101, 112, 127, 142, 159,
       172, 189, 202, 218, 229, 245, 240, 266, 279, 302, 316, 320, 338,
       347, 358, 366, 383, 381, 406, 418, 428, 439, 450, 463, 480, 501])

In [56]:
tuple([1,2])

(1, 2)